In [ ]:
from bs4 import BeautifulSoup
import requests
import os
import pandas as pd
import numpy as np
import pymongo


# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://127.0.0.1:27017/'
client = pymongo.MongoClient(conn)

# Define database and collection
db = client.beard_db
collection = db.awards

#check to see if insert many worked
documents = db.awards.find()
test_list =[]
for document in documents:
    test_list.append(document)
db_awards_df = pd.DataFrame(test_list)

# #This module takes all of our winning persons and randomly selects 100 unique authors for analysis. 
# import random

# winners_unique = db_awards_df.loc[db_awards_df['achievement status'] == "Winner",['honoree name']].value_counts()

# winner_list = []
# while len(winner_list) < 100:
#     random_index = random.randint(0,len(winners_unique)-1)
#     winner = winners_unique.index[random_index][0]
#     if winner in winner_list:
#         continue
#     else:
#         winner_list.append(winner)


# pd.DataFrame(winner_list).to_csv("C:/Users/JoshWeidenaar/Documents/Bootcamp/1_Homework/Project-2/listof100.csv")

db_awards_df["api_name"] = [i.split("/")[-1].replace("_","%20") for i in db_awards_df['wiki_url']]

honoree_names = db_awards_df['api_name'].value_counts().index.tolist()

# Wikiscrape Module


listie_dicts = []
for i in honoree_names:

    WikiMedia_URL = f"https://en.wikipedia.org/w/api.php?action=query&prop=pageprops&ppprop=wikibase_item&titles={i}&formatversion=2&format=json"
    response = requests.get(WikiMedia_URL).json()
    id_for_both = response['query']['pages'][0]
    missing_key = 'missing'
    found_key = 'pageprops'
    honoree_dict = {}
    honoree_dict['name'] = i
    if found_key in id_for_both.keys():
        found_ID = id_for_both['pageprops']['wikibase_item']
        honoree_dict['id'] = found_ID
    elif missing_key in id_for_both.keys():
        missing_ID = id_for_both['missing']
        honoree_dict['id'] = missing_ID
    else: 
        print(f"somethings wrong with {i}")
    listie_dicts.append(honoree_dict)
            

df = pd.DataFrame(listie_dicts).loc[pd.DataFrame(listie_dicts)['id'] != True,]

df = df.loc[pd.notna(df['id']),]

Sex_ID = "P21"
gender_possibilities = { "Q6581072" : 'Female',
 "Q6581097" : 'Male',
 "Q1097630" : 'Intersex',
 'Q1052281' : 'Transgender_Female',
 'Q2449503' : 'Transgender_Male'}

input_possibilities = list(gender_possibilities.values())

nums = [input_possibilities.index(i) for i in input_possibilities]

text_forinput = "0 == Female\n\n 1 == Male\n\n 2 == Intersex\n\n 3 == Transgender_Female\n\n 4 == Transgender_Male"

df.reset_index(inplace=True)

author_gender_list = []
for idx, i in enumerate(df['id']):
    print(f"author: {df['name'][idx]} \n{(idx/len(df))*100}% of the way there\n\n")
    WikiData_URL = f"https://m.wikidata.org/wiki/Special:EntityData/{i}.json"
    response = requests.get(WikiData_URL).json()
    try:
        property_verification = response['entities'][i]['claims'][Sex_ID][0]['mainsnak']['property']
        gender_found = response['entities'][i]['claims'][Sex_ID][0]['mainsnak']['datavalue']['value']['id']

        if gender_found in gender_possibilities.keys():
            gender_found = gender_possibilities[gender_found]
        else:
            gender_found = "NOTFOUND"
        author_gender_list.append(gender_found)
    except KeyError:
        print(f"{i} prompted a KeyError \n please enter gender assumed gender of {df['name'][idx]}\n\n\n{text_forinput} ")
        author_gender_list.append(input_possibilities[int(input())])

    

#change Jack Turner to MALE
df['gender'] = author_gender_list

df['matchname'] = [i.replace('%20'," ") for i in df['name']]

df_final = pd.merge(db_awards_df,df,how='left',left_on='honoree name',right_on='matchname')

len(df_final.loc[pd.notnull(df_final['gender']),'honoree name'].value_counts())

df_final.drop(axis=1,labels=['api_name','name','matchname','index'],inplace=True)

df_final.to_json('Beard_db_final.json', default_handler=str,orient="records")